In [ ]:
# Import necessary libraries
import tensorflow as tf  # TensorFlow for deep learning
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Image preprocessing and augmentation
from tensorflow.keras.applications import InceptionV3  # Pre-trained InceptionV3 model
from tensorflow.keras.models import Model  # Model creation
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D  # Layers for custom model
from tensorflow.keras.optimizers import Adam  # Optimizer
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping  # Callbacks for training
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay  # Evaluation metrics
import matplotlib.pyplot as plt  # Visualization
import pandas as pd  # For loading and processing CSV data
from tensorflow.keras.layers import BatchNormalization, LeakyReLU  # Additional layers

# Paths to the datasets
train_dir = '/kaggle/input/datathon-ai-qualification-round/train/train'
test_dir = '/kaggle/input/datathon-ai-qualification-round/test/test'
train_csv = '/kaggle/input/datathon-ai-qualification-round/train_data.csv'
test_csv = '/kaggle/input/datathon-ai-qualification-round/test.csv'

# Load train and test datasets from CSV files
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [ ]:
# Data augmentation with ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to range [0, 1]
    rotation_range=35,  # Random rotation by up to 35 degrees
    width_shift_range=0.3,  # Random horizontal shift
    height_shift_range=0.3,  # Random vertical shift
    shear_range=0.3,  # Shear transformation
    zoom_range=0.3,  # Random zoom
    horizontal_flip=True,  # Random horizontal flipping
    fill_mode='nearest',  # Fill mode for transformed pixels
    validation_split=0.2  # Split data into 80% training and 20% validation
)

# Train generator: Generates augmented images for training
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,  # Input DataFrame
    directory=train_dir,  # Directory containing training images
    x_col='filename',  # Column containing image filenames
    y_col='city',  # Column containing target labels
    target_size=(640, 640),  # Resize images to 640x640
    batch_size=32,  # Number of images per batch
    class_mode='categorical',  # Output labels are categorical
    subset='training'  # Use training subset
)

# Validation generator: Generates augmented images for validation
validation_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='filename',
    y_col='city',
    target_size=(640, 640),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use validation subset
)

# Test generator: Generates images for testing without labels
test_datagen = ImageDataGenerator(rescale=1.0/255)  # Normalize pixel values
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='filename',
    y_col=None,  # No labels in test data
    target_size=(640, 640),
    batch_size=32,
    class_mode=None,
    shuffle=False  # Do not shuffle test data
)

In [ ]:
# Load InceptionV3 pre-trained model without the top layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(640, 640, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce feature maps to a single vector
x = Dense(108, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))(x)  # Fully connected layer with L2 regularization
x = BatchNormalization()(x)  # Normalize activations
x = Dropout(0.5)(x)  # Dropout for reducing overfitting
output = Dense(3, activation='softmax')(x)  # Output layer with 3 classes (softmax activation)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers to preserve pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last 130 layers for fine-tuning
for layer in base_model.layers[-130:]:
    layer.trainable = True

# Compile the model with optimizer, loss, and metrics
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Adam optimizer with a small learning rate
    loss='categorical_crossentropy',  # Loss function for multi-class classification
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]  # Evaluation metrics
)

# Define callbacks
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)  # Reduce learning rate if no improvement
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)  # Stop training if no improvement

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,  # Train for 30 epochs
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

# Fine-tuning: Train the model further with a reduced learning rate
history_finetune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

In [ ]:
# Save the trained model
model.save("inceptionv3_optimized_model.h5")
print("Model kaydedildi: inceptionv3_optimized_model.h5")

# Plot training and validation loss
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'] + history_finetune.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'] + history_finetune.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'] + history_finetune.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'] + history_finetune.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Predict on the validation set
val_predictions = model.predict(validation_generator)
y_true = validation_generator.classes
y_pred = val_predictions.argmax(axis=1)

# Calculate the Macro F1 score
f1_macro = f1_score(y_true, y_pred, average='macro')
print(f"InceptionV3 Fine-Tuned Macro F1 Score: {f1_macro}")

# Predict on the test set
predictions = model.predict(test_generator)
predicted_classes = predictions.argmax(axis=1)

# Map predicted class indices to class names
label_map = {v: k for k, v in train_generator.class_indices.items()}
test_df['city'] = [label_map[class_idx] for class_idx in predicted_classes]

# Save predictions to a submission file
submission_path = '/kaggle/working/submission.csv'
test_df[['filename', 'city']].to_csv(submission_path, index=False)
print(f"Submission dosyası kaydedildi: {submission_path}")